In [1]:
# Importing libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

from pyspark.context import SparkContext
from pyspark.sql.context import SQLContext
from pyspark.sql.session import SparkSession

In [2]:

#sc = SparkContext()
sc=SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
spark = SparkSession(sc)

In [3]:
page1='https://www.simplyhired.com/search?q=big+data+engineer&l=United+States&job=ub4DHpuVXa22bbL-bGMvsmpBEYJyApC38ZwIH6V0WUpOav4oe5RZIA'
page2='https://www.simplyhired.com/search?q=big+data+engineer&l=united+states&sb=dd&pn=2&job=MrNtePJJR-mAmQM8Tiba5rnHB16pySp-UvaY0XdiWugpNfotFPsEtQ'
pagelist=[page1,page2]

mydf=[]
for i in pagelist:
        page = requests.get(i)
        soup = BeautifulSoup(page.content, 'html.parser') # Access on the whole web front-end content
        #print(soup)
        #print(soup.find_all('a'))# find all links

        job_container=soup.find(id="job-list")#fAccess on the job-list container
        #print(job_list)

        
        jobs=job_container.find_all(class_ ='SerpJob')#Accessing all of the job listed
        # for i in range(len(jobs)):
        #         print(jobs[i].find(class_='jobposting-title-container').get_text())#title,
        #         print(jobs[i].find(class_='JobPosting-labelWithIcon jobposting-company').get_text())#company
        #         print(jobs[i].find(class_='JobPosting-labelWithIcon jobposting-location').get_text())#location
        #         print(jobs[i].find(class_='jobposting-salary').get_text())#salary
        #         print(jobs[i].find(class_='jobposting-snippet').get_text())#description
        #         print(jobs[i].find(class_='SerpJob-timestamp').get_text())#date
        #         #print(jobs[i].find(SerpJob-timestamp.datetime))#.get_text())#
        #         #print(jobs[i].find_all('a'))# find all links      
        #         print()#datetime

        
        titles=[jobs[i].find(class_='jobposting-title-container').get_text() for i in range(len(jobs))]
        companies=[jobs[i].find(class_='JobPosting-labelWithIcon jobposting-company').get_text() for i in range(len(jobs))]
        locations=[jobs[i].find(class_='JobPosting-labelWithIcon jobposting-location').get_text() for i in range(len(jobs))]
        salaries=[jobs[i].find(class_='SerpJob-metaInfoLeft').get_text() for i in range(len(jobs))]#jobposting-salary
        descriptions=[jobs[i].find(class_='jobposting-snippet').get_text() for i in range(len(jobs))]
        timestamps=[jobs[i].find(class_='SerpJob-timestamp').get_text() for i in range(len(jobs))]
        links=['https://www.simplyhired.com'+jobs[i].find('a').get('href') for i in range(len(jobs))]#
        

        jobs_df=pd.DataFrame(
            {
              'titles':titles,  
              'companies':companies,
              'locations':locations,
              'salaries':salaries,
              'descriptions':descriptions,
              'timestamps':timestamps,
              'links':links

            })
        mydf.append(jobs_df)

#print(jobs_df) 
result = pd.concat(mydf)
print(links[6])



https://www.simplyhired.com/job/UUyVXAezLbE_UwtxmOqSa8yh92MbAd_oXQYV-9yKb-jbKveH9jfpDg?q=big+data+engineer


In [4]:
from pyspark.sql.types import *

#for i in range(len(mydf)):
mySchema = StructType([ StructField("titles", StringType(), True)\
                       ,StructField("companies", StringType(), True)\
                       ,StructField("locations", StringType(), True)\
                       ,StructField("salaries", StringType(), True)\
                       ,StructField("descriptions", StringType(), True)\
                       ,StructField("timestamps", StringType(), True)\
                       ,StructField("links", StringType(), True)])
df = spark.createDataFrame(result,schema=mySchema)

df.show(30)

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+
|              titles|           companies|           locations|            salaries|        descriptions|timestamps|               links|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+
|Big Data Engineer...|  iLink Systems Inc.|         Dallas, TX |$60 - $75 an hour...|Big Data Engineer...|        4d|https://www.simpl...|
|MongoDB Data Engi...|BlueCross BlueShi...|    Chattanooga, TN |Estimated: $65,00...|Ability to apply ...|          |https://www.simpl...|
|       Data Engineer|                GMAD|    Los Angeles, CA |$100,000 - $120,0...|Ideal candidate w...|          |https://www.simpl...|
|Senior Big Data E...|American Family I...|        Madison, WI |Estimated: $140,0...|R15759 Senior Big...|          |https://www.simpl...|
|       Data Engineer|  Equ

In [5]:
#add the jar to the spark/jar to link spark to mysql
df.write.format('jdbc').options(url='jdbc:mysql://mydb.cqdk5nbfyybo.us-east-2.rds.amazonaws.com:3306/mydb',driver='com.mysql.jdbc.Driver',dbtable='SIMPLYHIRED',user='admin',password='Password').mode('append').save()

In [6]:
#https://www.youtube.com/watch?v=E5cSNSeBhjw
#https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
#https://hackersandslackers.com/scraping-urls-with-beautifulsoup/
#https://opensource.com/article/19/5/log-data-apache-spark